In [1]:
import pandas as pd

import os, sys

def test_dir(dir):
    if not(os.path.isdir(dir)):
        os.mkdir(dir)
    return os.path.isdir(dir)

In [2]:
os.listdir('..//data//raw')

['.gitkeep', 'noaa_global', 'tech_challenge', 'temp_change', 'wbpy']

## Basic data from tech challenge

- [Dados da Vitivinicultura](http://vitibrasil.cnpuv.embrapa.br/index.php?opcao=opt_01) was indicated as main source of data;

### raw > interim

In [3]:
# Ajustando dataframe das exportações

#0. Create folder
test_dir('..//data//interim//tech_challenge//')

df_exp = pd.read_csv('..//data//raw//tech_challenge//ExpVinho.csv', sep=';')
df_exp = df_exp.melt(id_vars=['Id','País'])

vars = df_exp.variable.unique()
valor = [x for x in vars if x.endswith('.1')]
quantidade = list(set(vars) - set(valor))

df_quant = df_exp.loc[df_exp['variable'].isin(quantidade)]\
    .rename(columns={
          'value':'quantidade_exportada_pais'
        , 'variable':'ano'
        , 'País':'pais'
        , 'Id':'id'
        })

df_quant['ano'] = df_quant['ano'].astype(int)

df_value = df_exp.loc[df_exp['variable'].isin(valor)]\
    .rename(columns={
          'value':'valor_exportado_pais'
        , 'variable':'ano'
        , 'País':'pais'
        , 'Id':'id'
        })
df_value['ano'] = [int(x.replace('.1', '')) for x in df_value['ano']]

df_exp = pd.merge(df_quant, df_value, on=['id','pais','ano'])[['id','pais','ano','quantidade_exportada_pais','valor_exportado_pais']]\
    .drop(columns=['id'])


df_exp.to_csv('..//data//interim//tech_challenge//exportacao_vinhos.csv',index=False, sep=';', decimal=',') # export data to share with the project group members


In [4]:
# Ajustando df das comercializações no RS

#0. Create folder
test_dir('..//data//interim//tech_challenge//')

df_com = pd.read_csv('..//data//raw//tech_challenge//Comercio.csv', sep=';', header=None)
lista_anos = list(df_exp['ano'].unique())
df_com.columns = ['id','id_produto','produto'] + lista_anos

df_com = df_com\
    .melt(id_vars=['id','id_produto','produto'])\
    .rename(columns={
          'variable':'ano'
        , 'value':'quantidade_com_rs'
    })\
    .drop(columns=['id'])

df_com['id_produto'] = [str(x).strip().lower() for x in df_com['id_produto']]
df_com['produto'] = [str(x).strip().lower() for x in df_com['produto']]
df_com['ano'] = df_com['ano'].astype(int)

df_com.to_csv('..//data//interim//tech_challenge//comercio_vinhos_rs.csv',index=False, sep=';', decimal=',') # export data to share with the project group members

In [5]:
# Ajustando df das produções no RS

#0. Create folder
test_dir('..//data//interim//tech_challenge//')

df_prod = pd.read_csv('..//data//raw//tech_challenge//Producao.csv', sep=';', header=None)
lista_anos = list(df_exp['ano'].unique())
df_prod.columns = ['id','id_produto','produto'] + lista_anos

df_prod = df_prod\
    .melt(id_vars=['id','id_produto','produto'])\
    .rename(columns={
          'variable':'ano'
        , 'value':'quantidade_prod_rs'
    })\
    .drop(columns=['id'])

df_prod['id_produto'] = [str(x).strip().lower() for x in df_prod['id_produto']]
df_prod['produto'] = [str(x).strip().lower() for x in df_prod['produto']]
df_prod['ano'] = df_prod['ano'].astype(int)

df_prod.to_csv('..//data//interim//tech_challenge//producao_vinhos_rs.csv',index=False, sep=';', decimal=',') # export data to share with the project group members

### interim > processed

In [6]:
# interim to processed

#0. Create folder
test_dir('..//data//processed//tech_challenge//')

df_exp = pd.read_csv('..//data//interim//tech_challenge//exportacao_vinhos.csv', sep=';', decimal=',')
df_prod = pd.read_csv('..//data//interim//tech_challenge//producao_vinhos_rs.csv', sep=';', decimal=',')
df_com = pd.read_csv('..//data//interim//tech_challenge//comercio_vinhos_rs.csv', sep=';', decimal=',')

df_prod_com = df_prod.merge(df_com, on=['id_produto','produto','ano'], how='outer')

df_final = df_exp.merge(df_prod_com, on='ano', how='outer')

df_final.to_csv('..//data//processed//tech_challenge//df_vinhos.csv',index=False, sep=';', decimal=',') # export data to share with the project group members

## [Temperature Change over years](https://www.kaggle.com/datasets/sevgisarac/temperature-change)

### raw > processed


In [7]:
# Ajustar output da mudança de temperatura

#0. Create folder
test_dir('..//data//processed//temp_change//')

df = pd.read_csv("..//data//raw//temp_change//Environment_Temperature_change_E_All_Data_NOFLAG.csv", encoding='latin-1') # csv file is encoding as latin-1 type
df_countrycode=pd.read_csv('..//data//raw//temp_change//FAOSTAT_data_11-24-2020.csv') #this csv file includes ISO-3 Country Code, this mentioned in Data Wrangling 

#1. Renaming
df.rename(columns = {'Area':'Country Name'},inplace = True)
df.set_index('Months', inplace=True)
df.rename({'Dec\x96Jan\x96Feb': 'Winter', 'Mar\x96Apr\x96May': 'Spring', 'Jun\x96Jul\x96Aug':'Summer','Sep\x96Oct\x96Nov':'Fall'}, axis='index',inplace = True)
df.reset_index(inplace = True)

#2. Filtering 
df = df[df['Element'] == 'Temperature change']

#2. Drop unwanted columns from df_countrycode
df_countrycode.drop(['Country Code','M49 Code','ISO2 Code','Start Year','End Year'],axis=1,inplace=True)
df_countrycode.rename(columns = {'Country':'Country Name','ISO3 Code':'Country Code'},inplace=True)

#3. Merging with df to df_country
df = pd.merge(df, df_countrycode, how='outer', on='Country Name')

#2. Drop unwanted columns
df.drop(['Area Code','Months Code','Element Code','Element','Unit'],axis=1,inplace=True)

#3.Channing dataframe organization
df = df.melt(id_vars=["Country Code", "Country Name","Months",], var_name="year", value_name="tem_change")
df["year"] = [i.split("Y")[-1] for i in df.year]

df = df[df['Months']=='Meteorological year']# chose just year data
df.drop(['Months'],axis=1,inplace=True) # dropped Months column
df.to_csv('..//data//processed//temp_change//temperature_change_Data.csv',index=False, sep=';', decimal=',') # export data to share with the project group members

# [NOAA global data](https://www.kaggle.com/datasets/noaa/noaa-global-historical-climatology-network-daily)


### raw > interim

In [8]:
import numpy as np
import time
import os
import datetime
import pandas as pd
import multiprocessing as mp
import time

base_path = '..\\data\\raw\\noaa_global'
dest_path = '..\\data\\interim\\noaa_global'

In [9]:
test_dir(dest_path)

True

In [10]:
def process_year(file_year='2019'):
    base_path = '..\\data\\raw\\noaa_global'
    dest_path = '..\\data\\interim\\noaa_global'
    if not os.path.exists(f"{dest_path}\\{file_year}.csv"):

        start = datetime.datetime.now()

        raw_df = pd.read_csv(f"{base_path}\\ghcnd_all_years\\{file_year}.csv.gz",
                        usecols=[0,1,2,3], 
                        names=['station_id','date', 'stat', 'value'], 
                        dtype= {
                              'station_id' : str
                            , 'date': str
                            , 'stat': str
                            , 'value': np.int16
                            },
                        engine='c'
                        )
        raw_df['year'] = np.int16(file_year)
        grouped = raw_df.groupby(["station_id", 'year',"stat"]).mean().reset_index()
        grouped = grouped[['year', 'station_id', 'stat', 'value']]
        #return grouped
        grouped.to_csv(f"{dest_path}\\{file_year}.csv", index=False)
        duration = (datetime.datetime.now() - start).seconds
        print(f"{file_year} took {round(duration/60, 2)} minutes.")

In [11]:
years = [year[:4] for year in os.listdir(f"{base_path}\\ghcnd_all_years\\")]
years.sort()
level_of_parallelism = mp.cpu_count()
pool = mp.Pool(level_of_parallelism)
pool.map(process_year, years)

In [ ]:
df_stations = pd.read_csv(base_path+'\\ghcnd-stations.txt', sep=';', decimal='.')
for col in ['id','name']:
    df_stations[col] = [x.strip() for x in df_stations[col]]
df_stations.to_csv(dest_path+'\\df_stations.csv',index=False, sep=';', decimal=',')

### interim > processed

In [13]:
import pandas as pd
import numpy as np
import os

In [6]:
base_path = '..\\data\\interim\\noaa_global\\'
dest_path = '..\\data\\processed\\noaa_global\\'

In [7]:
files = [base_path + x for x in os.listdir(base_path)]

In [14]:
df_noaa = pd.read_csv(base_path+"*",
                      dtype= {
                              'station_id' : str
                            , 'date': str
                            , 'stat': str
                            , 'value_min' : np.float64
                            , 'value_mean' : np.float64
                            , 'value_median' : np.float64
                            , 'value_max' : np.float64
                            },
                        engine='c'
                      )

ValueError: cannot safely convert passed user dtype of int16 for float64 dtyped data in column 3

In [11]:
df_noaa

,year,station_id,stat,value
0,1763,ITE00100554,TMAX,147.873973
1,1763,ITE00100554,TMIN,100.657534


### interim > processed

In [30]:
import os, glob

base_path = '..\\data\\interim\\noaa_global\\'

def read_csv(args):
    return pd.read_csv(args, sep=';', decimal='.')

df_noaa = pd.concat(map(read_csv, glob.glob(base_path+'years\\*.csv')))

df_stations = pd.read_csv(base_path+'df_stations.csv', sep=';', decimal=',').rename(columns={'id':'station_id'})

df_noaa = df_noaa.merge(df_stations)

,station_id,year,stat,value_min,value_mean,value_median,value_max,latitude,longitude,elevation,name
0,ITE00100554,1763,TMAX,-39,147.873973,147.0,309,45.4717,9.1892,150.0,MILAN
1,ITE00100554,1763,TMIN,-63,100.657534,99.0,233,45.4717,9.1892,150.0,MILAN
2,ITE00100554,1764,TMAX,20,153.592896,148.5,301,45.4717,9.1892,150.0,MILAN
3,ITE00100554,1764,TMIN,-19,106.581967,93.0,236,45.4717,9.1892,150.0,MILAN
4,ITE00100554,1765,TMAX,-14,149.994521,157.0,292,45.4717,9.1892,150.0,MILAN
...,...,...,...,...,...,...,...,...,...,...,...
12143427,USC00453859,2019,DAPR,3,3.250000,3.0,4,47.3872,-121.3925,776.9,WA HYAK
12143428,USC00453859,2019,MDPR,38,189.250000,212.0,295,47.3872,-121.3925,776.9,WA HYAK
12143429,USC00453859,2019,PRCP,0,59.000000,3.0,516,47.3872,-121.3925,776.9,WA HYAK
12143430,USC00453859,2019,SNOW,0,103.526316,10.0,711,47.3872,-121.3925,776.9,WA HYAK
